In [10]:
import pandas as pd
from datetime import datetime
import numpy as np

from pandas_datareader import data as pdr
# import fix_yahoo_finance as yf
from datetime import datetime

In [12]:
stock_list = ['AMZN', 'MSFT', 'AAPL']
stock_dict = {}
for stock in stock_list:
    dim = (0,0)
    while dim != (1,6):
       s = pdr.get_data_yahoo(stock, start="2017-08-13",end="2017-08-14")
       dim = s.shape
    stock_dict[stock] = s

In [14]:
df1 = pdr.get_data_yahoo("IBM", 
                         start=datetime(1970, 1, 1), 
                         end=datetime.today())
df1.rename(columns={'Adj Close': 'ibm'}, inplace=True)

df2 = pdr.get_data_yahoo("F", 
                         start=datetime(1970, 1, 1), 
                         end=datetime.today())
df2.rename(columns={'Adj Close': 'ford'}, inplace=True)

df = df1.join(df2.ford, how='inner')

In [16]:
del df["Open"]
del df["High"]
del df["Low"]
del df["Close"]
del df["Volume"]

In [18]:
df

,ibm,ford
Date,,
1972-06-01,1.696454,0.002854
1972-06-02,1.694328,0.002848
1972-06-05,1.677321,0.002848
1972-06-06,1.670943,0.002816
1972-06-07,1.680509,0.002800
...,...,...
2020-05-04,120.069717,4.860000
2020-05-05,120.957809,4.970000
2020-05-06,121.540001,4.870000


In [21]:
import datetime as DT
import numpy as np
import pandas as pd
# import pandas.io.data as PID
from pandas_datareader import data as pdr

def setup_df():
    df1 = pdr.get_data_yahoo("IBM", 
                             start=DT.datetime(1970, 1, 1), 
                             end=DT.datetime.today())
    df1.rename(columns={'Adj Close': 'ibm'}, inplace=True)

    df2 = pdr.get_data_yahoo("F", 
                             start=DT.datetime(1970, 1, 1), 
                             end=DT.datetime.today())
    df2.rename(columns={'Adj Close': 'ford'}, inplace=True)

    df = df1.join(df2.ford, how='inner')
    df = df[['ibm', 'ford']]
    df['sh ibm'] = 0
    df['sh ford'] = 0
    df['ibm value'] = 0
    df['ford value'] = 0
    df['ratio'] = 0
    return df

def invest(df, i, amount):
    """
    Invest amount dollars evenly between ibm and ford
    starting at ordinal index i.
    This modifies df.
    """
    c = dict([(col, j) for j, col in enumerate(df.columns)])
    halfvalue = amount/2
    df.iloc[i:, c['sh ibm']] = halfvalue / df.iloc[i, c['ibm']]
    df.iloc[i:, c['sh ford']] = halfvalue / df.iloc[i, c['ford']]

    df.iloc[i:, c['ibm value']] = (
        df.iloc[i:, c['ibm']] * df.iloc[i:, c['sh ibm']])
    df.iloc[i:, c['ford value']] = (
        df.iloc[i:, c['ford']] * df.iloc[i:, c['sh ford']])
    df.iloc[i:, c['ratio']] = (
        df.iloc[i:, c['ibm value']] / df.iloc[i:, c['ford value']])

def rebalance(df, tol, i=0):
    """
    Rebalance df whenever the ratio falls outside the tolerance range.
    This modifies df.
    """
    c = dict([(col, j) for j, col in enumerate(df.columns)])
    while True:
        mask = (df['ratio'] >= 1+tol) | (df['ratio'] <= 1-tol)
        # ignore prior locations where the ratio falls outside tol range
        mask[:i] = False
        try:
            # Move i one index past the first index where mask is True
            # Note that this means the ratio at i will remain outside tol range
            i = np.where(mask)[0][0] + 1
        except IndexError:
            break
        amount = (df.iloc[i, c['ibm value']] + df.iloc[i, c['ford value']])
        invest(df, i, amount)
    return df

In [32]:
df = setup_df()
tol = 0.05
invest(df, i=0, amount=100)

df

,ibm,ford,sh ibm,sh ford,ibm value,ford value,ratio
Date,,,,,,,
1972-06-01,1.696454,0.265154,29.473244,188.569383,50.000000,50.000000,1.000000
1972-06-02,1.694328,0.264655,29.473244,188.569383,49.937344,49.905835,1.000631
1972-06-05,1.677321,0.264655,29.473244,188.569383,49.436079,49.905835,0.990587
1972-06-06,1.670943,0.261659,29.473244,188.569383,49.248104,49.340859,0.998120
1972-06-07,1.680509,0.260161,29.473244,188.569383,49.530054,49.058368,1.009615
...,...,...,...,...,...,...,...
2020-05-04,120.069717,4.860000,29.473244,188.569383,3538.844037,916.447226,3.861482
2020-05-05,120.957809,4.970000,29.473244,188.569383,3565.018990,937.189794,3.803946
2020-05-06,121.540001,4.870000,29.473244,188.569383,3582.178061,918.332873,3.900740


In [23]:
df = setup_df()
tol = 0.05
invest(df, i=0, amount=100)
rebalance(df, tol)

,ibm,ford,sh ibm,sh ford,ibm value,ford value,ratio
Date,,,,,,,
1972-06-01,1.696454,0.265154,29.473244,188.569383,50.000000,50.000000,1.000000
1972-06-02,1.694328,0.264655,29.473244,188.569383,49.937344,49.905835,1.000631
1972-06-05,1.677321,0.264655,29.473244,188.569383,49.436079,49.905835,0.990587
1972-06-06,1.670943,0.261659,29.473244,188.569383,49.248104,49.340859,0.998120
1972-06-07,1.680509,0.260161,29.473244,188.569383,49.530054,49.058368,1.009615
...,...,...,...,...,...,...,...
2020-05-04,120.069717,4.860000,32.229671,778.477198,3869.807446,3783.399288,1.022839
2020-05-05,120.957809,4.970000,32.229671,778.477198,3898.430360,3869.031513,1.007599
2020-05-06,121.540001,4.870000,32.229671,778.477198,3917.194199,3791.183867,1.033238


In [29]:
df['portfolio value'] = df['ibm value'] + df['ford value']
df['ibm weight'] = df['ibm value'] / df['portfolio value']
df['ford weight'] = df['ford value'] / df['portfolio value']

print(df['ibm weight'].min())
print(df['ibm weight'].max())
print(df['ford weight'].min())
print(df['ford weight'].max())

# This shows the rows which trigger rebalancing
mask = (df['ratio'] >= 1+tol) | (df['ratio'] <= 1-tol)
print(df.loc[mask])

0.44400340006332073
0.5569251597025968
0.4430748402974033
0.5559965999366792


In [30]:
# This shows the rows which trigger rebalancing
mask = (df['ratio'] >= 1+tol) | (df['ratio'] <= 1-tol)
print(df.loc[mask])

                   ibm      ford     sh ibm     sh ford    ibm value  \
Date                                                                   
1972-07-05    1.700706  0.252670  29.473244  188.569383    50.125315   
1972-08-23    1.761794  0.280563  28.778776  193.232322    50.702267   
1972-09-21    1.704205  0.255333  29.768111  187.016863    50.730962   
1972-10-11    1.666878  0.265929  29.049039  191.763730    48.421201   
1972-11-02    1.667946  0.283978  29.951915  186.152323    49.958162   
...                ...       ...        ...         ...          ...   
2020-03-26  111.396042  5.250000  35.813892  694.013515  3989.525796   
2020-03-30  111.435516  5.030000  34.801496  714.807684  3878.122697   
2020-04-02  108.544289  4.360000  33.171160  751.755895  3600.539939   
2020-04-08  117.711349  5.030000  31.773620  786.342610  3740.115676   
2020-04-20  118.816528  4.980000  33.497059  747.864625  3980.004292   

             ford value     ratio     pf value  portfolio value